In [81]:
# Importa as bibliotecas
import pandas as pd
import numpy as np
import os

In [82]:
# Verifica a pasta corrente
pasta = os.getcwd()

In [83]:
# Junta caminho corrente + pasta com os arquivos "ambos", "homens", "mulheres"
pasta_ambos = os.path.join(pasta, "ambos")
pasta_mulheres = os.path.join(pasta, "mulheres")
pasta_homens = os.path.join(pasta, "homens")

In [84]:
# Lista arquivos das pastas
arquivos_ambos = os.listdir(pasta_ambos)
arquivos_mulheres = os.listdir(pasta_mulheres)
arquivos_homens = os.listdir(pasta_homens)

In [85]:
# Lista os arquivos somente excel e que comecem com 'ambos', 'homens', 'mulheres'
arq_ambos_xls = [arq_ambos for arq_ambos in arquivos_ambos if arq_ambos[-3:]=='xls']
arq_mulher_xls = [arq_mulher for arq_mulher in arquivos_mulheres if arq_mulher[-3:]=='xls']
arq_homem_xls = [arq_homem for arq_homem in arquivos_homens if arq_homem[-3:]=='xls']

In [86]:
len(arq_ambos_xls)

22

Obs.:
> Pula_linhas: Existe um modelo em todos os arquivos. Não imprimir algumas linhas
> assign(arq), para gravar o ano da tabela no dataframe
#### colunas:
1. x: idade da pessoa
2. qx_mil: probabilidade de morte entre duas idades exatas na idade x;
3. dx: número de mortos na idade x;
4. lx: número de vivos na idade x;
5. Lx: Pessoas vivas na metade da idade x;
6. Tx: Quantidade de existencia na idade x. Representa o somatório dos anos vividos pelas pessoas componentes de um grupo de idade x até o instante de sua extinção. Em outras palavras, é o acompanhamento, ano a ano, de um grupo até sua extinção;
7. Ex: Expectativa de vida na idade x.
8. qx: probabilidade de morte na idade x. corresponde qx_mil/1000

### Tratamento de inconsistencias na importação

In [7]:
# Inicializa o dataframe vazio
df_ambos = pd.DataFrame()

# Dataframe ambos
pular_linhas = list(range(0,5)) + list(range(46,62)) + list(range(103,113))
colunas = ['x', 'qx_mil', 'dx', 'lx', 'Lx', 'Tx', 'Ex']

for arq_ambos in arq_ambos_xls:
    ano = arq_ambos[-8:-4]
    dados=pd.read_excel(os.path.join(pasta_ambos, arq_ambos),
                        names = colunas,
                        skiprows = pular_linhas,
                        usecols = "A:G").assign(Ano=ano)
    df_ambos = df_ambos.append(dados)

In [8]:
# Inicializa o dataframe vazio
df_mulher = pd.DataFrame()

# Dataframe mulher
pular_linhas1 = list(range(0,5)) + list(range(46,62)) + list(range(103,113))
pular_linhas2 = list(range(0,5)) + list(range(46,61)) + list(range(102,113))
pular_linhas3 = list(range(0,4)) + list(range(45,61)) + list(range(102,113))
# 2003: pular_linhas2 = list(range(0,4)) + list(range(45,61)) + list(range(101,113))
colunas = ['x', 'qx_mil', 'dx', 'lx', 'Lx', 'Tx', 'Ex']

for arq_mulher in arq_mulher_xls:
    ano = arq_mulher[-8:-4]
    if ano in ['1998', '1999']: 
        pular_linhas = pular_linhas2
    elif ano in ['2003']:
        pular_linhas = pular_linhas3
    else:
        pular_linhas = pular_linhas1
    dados=pd.read_excel(os.path.join(pasta_mulheres, arq_mulher),
                        names = colunas,
                        skiprows = pular_linhas,
                        usecols = "A:G").assign(Ano=ano)
    df_mulher = df_mulher.append(dados)

In [9]:
# Inicializa o dataframe vazio
df_homem = pd.DataFrame()

# Dataframe ambos
pular_linhas1 = list(range(0,5)) + list(range(46,62)) + list(range(103,113))
pular_linhas2 = list(range(0,5)) + list(range(46,61)) + list(range(102,113))
colunas = ['x', 'qx_mil', 'dx', 'lx', 'Lx', 'Tx', 'Ex']

for arq_homem in arq_homem_xls:
    ano = arq_homem[-8:-4]
    if ano in ['1998', '1999']:
        pular_linhas = pular_linhas2
    else:
        pular_linhas = pular_linhas1
    
    dados=pd.read_excel(os.path.join(pasta_homens, arq_homem),
                        names = colunas,
                        skiprows = pular_linhas,
                        usecols = "A:G").assign(Ano=ano)
    df_homem = df_homem.append(dados)

In [10]:
df_ambos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 80
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       1701 non-null   object 
 1   qx_mil  1701 non-null   float64
 2   dx      1701 non-null   float64
 3   lx      1701 non-null   float64
 4   Lx      1701 non-null   float64
 5   Tx      1701 non-null   float64
 6   Ex      1701 non-null   float64
 7   Ano     1701 non-null   object 
dtypes: float64(6), object(2)
memory usage: 119.6+ KB


In [11]:
df_mulher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 80
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       1701 non-null   object 
 1   qx_mil  1701 non-null   float64
 2   dx      1701 non-null   float64
 3   lx      1701 non-null   float64
 4   Lx      1701 non-null   float64
 5   Tx      1701 non-null   float64
 6   Ex      1701 non-null   float64
 7   Ano     1701 non-null   object 
dtypes: float64(6), object(2)
memory usage: 119.6+ KB


In [12]:
df_homem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 80
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       1701 non-null   object 
 1   qx_mil  1701 non-null   float64
 2   dx      1701 non-null   float64
 3   lx      1701 non-null   float64
 4   Lx      1701 non-null   float64
 5   Tx      1701 non-null   float64
 6   Ex      1701 non-null   float64
 7   Ano     1701 non-null   object 
dtypes: float64(6), object(2)
memory usage: 119.6+ KB


#### Tratamento arquivo ambos

In [77]:
# Retirar sinais na idade 80 anos e colocar 80
# Tratamento da idade 80+ para os anos.
df_ambos[df_ambos['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,46279.693590,46279.693590,400621.693813,400621.693813,8.656533,2011
80,80,1000.0,40771.368891,40771.368891,379248.000000,379248.000000,9.301822,2004
80,80,1000.0,37399.000000,37399.000000,326226.000000,326226.000000,8.722854,1998
80,80,1000.0,38025.000000,38025.000000,339181.000000,339181.000000,8.919947,1999
80,80,1000.0,50974.035905,50974.035905,491557.682309,491557.682309,9.643295,2018


In [14]:
# Tratamento da idade 80+ para os anos.
df_ambos.loc[80,'x'] = 80
# Aproveitar e corrigir o qx aos 80 anos de alguns arquivos
df_ambos.loc[80,'qx_mil'] = 1000.0

In [78]:
df_ambos[df_ambos['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,46279.693590,46279.693590,400621.693813,400621.693813,8.656533,2011
80,80,1000.0,40771.368891,40771.368891,379248.000000,379248.000000,9.301822,2004
80,80,1000.0,37399.000000,37399.000000,326226.000000,326226.000000,8.722854,1998
80,80,1000.0,38025.000000,38025.000000,339181.000000,339181.000000,8.919947,1999
80,80,1000.0,50974.035905,50974.035905,491557.682309,491557.682309,9.643295,2018


#### Arquivo "ambos" pronto para a interpolação

#### Tratamento arquivo mulher

In [87]:
# Retirar sinais na idade 80 anos e colocar 80
# Tratamento da idade 80+ para os anos.
df_mulher[df_mulher['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,41560.000000,41560.000000,273882.000000,273882.000000,6.590000,1999
80,80,1000.0,47270.000000,47270.000000,449967.000000,449967.000000,9.519082,2001
80,80,1000.0,47851.000000,47851.000000,458151.000000,458151.000000,9.574533,2002
80,80,1000.0,59538.071914,59538.071914,616962.715702,616962.715702,10.362491,2018
80,80,1000.0,57198.845238,57198.845238,568987.880875,568987.880875,9.947541,2014


In [17]:
# Tratamento da idade 80+ para os anos.
df_mulher.loc[80,'x'] = 80
# Aproveitar e corrigir o qx aos 80 anos de alguns arquivos
df_mulher.loc[80,'qx_mil'] = 1000.0

In [88]:
df_mulher[df_mulher['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,41560.000000,41560.000000,273882.000000,273882.000000,6.590000,1999
80,80,1000.0,47270.000000,47270.000000,449967.000000,449967.000000,9.519082,2001
80,80,1000.0,47851.000000,47851.000000,458151.000000,458151.000000,9.574533,2002
80,80,1000.0,59538.071914,59538.071914,616962.715702,616962.715702,10.362491,2018
80,80,1000.0,57198.845238,57198.845238,568987.880875,568987.880875,9.947541,2014


#### Arquivo "mulher" pronto para a interpolação

#### Tratamento arquivo homem

In [89]:
# Retirar sinais na idade 80 anos e colocar 80
# Tratamento da idade 80+ para os anos.
df_homem[df_homem['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,37530.000020,37530.000020,300761.000000,300761.000000,8.013882,2011
80,80,1000.0,38426.589568,38426.589568,315031.317690,315031.317690,8.198264,2012
80,80,1000.0,31984.000000,31984.000000,279860.000000,279860.000000,8.750000,2002
80,80,1000.0,31179.000000,31179.000000,271022.000000,271022.000000,8.692453,2000
80,80,1000.0,41775.975201,41775.975201,357473.785439,357473.785439,8.556923,2017


In [20]:
# Tratamento da idade 80+ para os anos.
df_homem.loc[80,'x'] = 80
# Aproveitar e corrigir o qx aos 80 anos de alguns arquivos
df_homem.loc[80,'qx_mil'] = 1000.0

In [90]:
df_homem[df_homem['x'].astype(str).str.contains("80")].head()

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,37530.000020,37530.000020,300761.000000,300761.000000,8.013882,2011
80,80,1000.0,38426.589568,38426.589568,315031.317690,315031.317690,8.198264,2012
80,80,1000.0,31984.000000,31984.000000,279860.000000,279860.000000,8.750000,2002
80,80,1000.0,31179.000000,31179.000000,271022.000000,271022.000000,8.692453,2000
80,80,1000.0,41775.975201,41775.975201,357473.785439,357473.785439,8.556923,2017


#### Arquivo "homem" pronto para a interpolação

# fazer os gráficos para a escolher logo o dataset

### Interpolação para idades acima de 80 anos, inclusive.

#### Seguir os passos da Nota Técnica Atuarial sobre a Metodologia adotada pelo Ministério da Previdência Social na Extrapolação das Tábuas de Mortalidade IBGE para as idades acima de 80 anos
Fonte:https://www.example.com

In [113]:
# Para classificar por ano
#linha_anterior = sorted(df_ambos.loc[79,:].values.tolist(), key=lambda x: x[7])
#novas_instancias = sorted(df_ambos.loc[80,:].values.tolist(), key = lambda x: x[7])
# sorted --> subslistas classificadas pelo atributo "Ano" --> x[7]
df_to_lista = sorted(df_ambos.values.tolist(), key = lambda x: x[7])
novas_instancias = []
#ano = 2015 para testar 2015, i=17
fx = 0.5

#colunas = list(df_teste)
ano = 1997
for i in range(0,21):
    print(str(ano))
    ano += 1
    idade = 0
    FA = 100
    for idade in range(80,90): #colocar um stop de 150 para o range
        if idade==80:
            qx_mil = (df_to_lista[idade+81*i][2])/(df_to_lista[idade+81*i][3])*1000
            lx = df_to_lista[idade+81*i][3]
            lx1 = lx * (lx /(df_to_lista[idade+81*i-2][3]+FA))  
            dx = lx - lx1
            Lx = fx*lx + (1-fx)*lx1
            Tx = df_to_lista[idade+81*i][5]
            exp = Tx/lx
            valores = [idade, qx_mil, dx, lx, Lx, Tx, exp, str(ano)]
            df_to_lista[0:8] = valores
        else:
            qx_mil = (df_to_lista[idade+81*i][2])/(df_to_lista[idade+81*i][3])*1000
            lx = df_to_lista[idade+81*i-1][3] * (df_to_lista[idade+81*i-1][3] /(df_to_lista[idade+81*i-2][3]+FA))
            lx1 = lx * (lx /(df_to_lista[idade+81*i-2][3]+FA))
            dx = lx - lx1
            Lx = fx * lx + (1-fx)*lx1
            Tx = df_to_lista[idade+81*i][5] - Lx
            Exp = Tx/lx
            valores = [idade, qx_mil, dx, lx, Lx, Tx, exp, str(ano)]
            df_to_lista.extend(valores)
            print(ano, idade)
        

# os valores são os resultados das variáveis
#valores = [idade, qx_mil, dx, lx, Lx, Tx, Exp]
#zipped = zip(colunas, valores)
#a_dicionario = dict(zipped)
#print(a_dicionario)
#novos_dados.append(a_dicionario)
#df_ambos_add.append(a_dicionario)
#n+=1
#idade+=1


#print(qx_mil)
#print(lx)
#print(df_to_lista[80+81*17][2])
#print(df_to_lista[80+81*17][3])

#df_teste = df_teste.append(df_ambos_add, True)
#print(df)


1997
1998 81
1998 82
1998 83
1998 84
1998 85
1998 86
1998 87
1998 88
1998 89
1998
1999 81
1999 82
1999 83
1999 84
1999 85
1999 86
1999 87
1999 88
1999 89
1999
2000 81
2000 82
2000 83
2000 84
2000 85
2000 86
2000 87
2000 88
2000 89
2000
2001 81
2001 82
2001 83
2001 84
2001 85
2001 86
2001 87
2001 88
2001 89
2001
2002 81
2002 82
2002 83
2002 84
2002 85
2002 86
2002 87
2002 88
2002 89
2002
2003 81
2003 82
2003 83
2003 84
2003 85
2003 86
2003 87
2003 88
2003 89
2003
2004 81
2004 82
2004 83
2004 84
2004 85
2004 86
2004 87
2004 88
2004 89
2004
2005 81
2005 82
2005 83
2005 84
2005 85
2005 86
2005 87
2005 88
2005 89
2005
2006 81
2006 82
2006 83
2006 84
2006 85
2006 86
2006 87
2006 88
2006 89
2006
2007 81
2007 82
2007 83
2007 84
2007 85
2007 86
2007 87
2007 88
2007 89
2007
2008 81
2008 82
2008 83
2008 84
2008 85
2008 86
2008 87
2008 88
2008 89
2008
2009 81
2009 82
2009 83
2009 84
2009 85
2009 86
2009 87
2009 88
2009 89
2009
2010 81
2010 82
2010 83
2010 84
2010 85
2010 86
2010 87
2010 88
2010 89

TypeError: 'int' object is not subscriptable

In [105]:
#sorted(df_to_lista, key = lambda x: x[7])
df_to_lista

[80,
 1000.0,
 4906.633653045887,
 50974.03590466877,
 48520.719078145834,
 491557.6823086747,
 9.643295328389964,
 2018,
 [8,
  0.317113916626319,
  30.364000000000004,
  95751.08,
  95735.898,
  6196356.522875723,
  64.71317632005533,
  '1998'],
 [9,
  0.27910363729414644,
  26.715999999999998,
  95720.716,
  95707.35800000001,
  6100620.624875723,
  63.733545671301954,
  '1998'],
 [10,
  0.2845737454803855,
  27.232000000000006,
  95694.0,
  95680.38399999999,
  6004913.266875723,
  62.7511993110929,
  '1998'],
 [11,
  0.33122264567357396,
  31.687,
  95666.768,
  95650.9245,
  5909232.882875724,
  61.76891941071663,
  '1998'],
 [12,
  0.4315571186686192,
  41.272000000000006,
  95635.08099999999,
  95614.44499999999,
  5813581.958375724,
  60.789219788246164,
  '1998'],
 [13,
  0.5723383195244371,
  54.712,
  95593.809,
  95566.453,
  5717967.513375724,
  59.81524926343006,
  '1998'],
 [14,
  0.750645570786586,
  71.71600000000001,
  95539.097,
  95503.239,
  5622401.060375724,
  5

In [ ]:
# Arquivo ambos
# Interpolar features qx_mil, dx, Lx
# As features lx e Tx tem o mesmo valor na idade 80
# Ex (expectativa de vida) será a feature alvo para o solver. 
# Em outras palavras, a Ex na idade 80 deve tender ao mesmo valor antes do solver
# Criar feature qx depois dos laços
# Calcular o f0 = (Lx - lx+1)/dx.
# FA = Fator de Ajuste. Começar com 100
# fx = 0.5 ???????
# Diferenca = minimizar. Tender a 0. Meta do solver. diferença = Exp80_ant - Exp80calc

df_teste = df_ambos.copy()

# è melhor iterar sobre listas, criar um dicionário, onde as
# 1 - colunas serão a chave e e a coluna de dados como valor usando zip(keys, values)
# 2 - Criar um dicionário iterando sobre o zip usando dict(zipped)
# 3 - Armazena o dicionário criado em uma lista
# 4 - Append a lista de dicionários, chamaado dados, a um DataFrame
# existente usando pandas.DataFrame.append(data, ignore_index=None)
# Onde ignore_index como True, preserva os índices do DataFrame
colunas = list(df_teste)
#df_ambos_add = df_ambos.loc[80,:].copy().tolist()

# Linha anterior
linha_79 = sorted(df_ambos.loc[79,:].values.tolist(), key = lambda x: x[7])

# lista de dados em sublistas com os dados no indice = 80, classificado pela 7 coluna = ano
novos_dados = sorted(df_ambos.loc[80,:].values.tolist(), key = lambda x: x[7])

# linha_anterior[0][0] -> idade
# linha_anterior[0][]
# Primeira dimensão: Ano a ano aos 79 anos; 
# Segunda dimensão: Leitura na sublista na idade x
fx = 0.5
anos = range[1998,2019]
for ano in anos:
    FA = 100
    #data = []
    idade = 80 # idade inicial de interpolação
    n=0
    while n<20:
        qx_mil = (novos_dados[ano-1998][2])/(novos_dados[ano-1998][3])*1000      # (dx/lx)*1000
        dx = lx - lx+1
        lx = lx-1 - dx-1
        Lx = fx * lx + (1-fx)*lx
        Tx = soma(Lx)
        Exp = Tx/lx
        # os valores são os resultados das variáveis
        valores = [idade, qx_mil, dx, lx, Lx, Tx, Exp]
        zipped = zip(colunas, valores)
        a_dicionario = dict(zipped)
        print(a_dicionario)
        novos_dados.append(a_dicionario)
        #df_ambos_add.append(a_dicionario)
        n+=1
        idade+=1

        
df_teste = df_teste.append(df_ambos_add, True)
print(df)
#df_ambos_add.append()
#df_ambos_add.head()

In [22]:
#df_ambos_add = pd.DataFrame()
df_ambos_add = df_ambos[80:81]
df_ambos_add

,x,qx_mil,dx,lx,Lx,Tx,Ex,Ano
80,80,1000.0,46279.69359,46279.69359,400621.693813,400621.693813,8.656533,2011


In [23]:
type(df_ambos_add)

pandas.core.frame.DataFrame

In [24]:
sorted(df_ambos.loc[80,:].values.tolist(), key = lambda x: x[7])

[[80, 1000.0, 37399.0, 37399.0, 326226.0, 326226.0, 8.722853552234012, '1998'],
 [80, 1000.0, 38025.0, 38025.0, 339181.0, 339181.0, 8.919947403024326, '1999'],
 [80, 1000.0, 38688.0, 38688.0, 353052.0, 353052.0, 9.125620347394538, '2000'],
 [80, 1000.0, 39183.0, 39183.0, 359287.0, 359287.0, 9.169461245948495, '2001'],
 [80,
  1000.0,
  39683.99999999995,
  39683.99999999995,
  365635.0,
  365635.0,
  9.213662937203923,
  '2002'],
 [80,
  1000.0,
  40234.094969744845,
  40234.094969744845,
  372431.343947492,
  372431.343947492,
  9.256610450105867,
  '2003'],
 [80,
  1000.0,
  40771.36889142962,
  40771.36889142962,
  379248.0,
  379248.0,
  9.301821604516206,
  '2004'],
 [80,
  1000.0,
  41279.796194029,
  41279.796194029,
  385829.2952097976,
  385829.2952097976,
  9.34668604942402,
  '2005'],
 [80,
  1000.0,
  41982.0,
  41849.20092325195,
  393247.7368313408,
  393247.7368313408,
  9.396780061643838,
  '2006'],
 [80,
  1000.0,
  42411.185105963625,
  42411.185105963625,
  400707.82

In [25]:
df_ambos.loc[80,:].values.tolist()  #[]

[[80,
  1000.0,
  46279.69358971874,
  46279.69358971874,
  400621.69381346024,
  400621.69381346024,
  8.656532978914543,
  '2011'],
 [80,
  1000.0,
  40771.36889142962,
  40771.36889142962,
  379248.0,
  379248.0,
  9.301821604516206,
  '2004'],
 [80, 1000.0, 37399.0, 37399.0, 326226.0, 326226.0, 8.722853552234012, '1998'],
 [80, 1000.0, 38025.0, 38025.0, 339181.0, 339181.0, 8.919947403024326, '1999'],
 [80,
  1000.0,
  50974.03590466877,
  50974.03590466877,
  491557.6823086747,
  491557.6823086747,
  9.643295328389964,
  '2018'],
 [80,
  1000.0,
  41982.0,
  41849.20092325195,
  393247.7368313408,
  393247.7368313408,
  9.396780061643838,
  '2006'],
 [80,
  1000.0,
  47820.69392010142,
  47820.69392010142,
  439480.5856568577,
  439480.5856568577,
  9.190175834569438,
  '2013'],
 [80,
  1000.0,
  49775.834926876014,
  49775.834926876014,
  471431.77821492776,
  471431.77821492776,
  9.47109734889414,
  '2016'],
 [80, 1000.0, 38688.0, 38688.0, 353052.0, 353052.0, 9.125620347394538, 

In [26]:
linha_anterior = sorted(df_ambos.loc[79,:].values.tolist(), key=lambda x: x[7])
linha_anterior[0][7]

'1998'

In [31]:
df_to_lista = sorted(df_ambos.values.tolist(), key = lambda x: x[7])
df_to_lista

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [2,
  1.9100808670106195,
  184.0,
  96331.0,
  96239.0,
  6772088.110875723,
  70.30019527333593,
  '1998'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998'],
 [8,
  0.317113916626319,
  30.364000000000004,
  95751.08,
  95735.898,
  6196356.522875723,
  64.71

In [74]:
df_to_lista[323][7]  # [linha][coluna] 80-1998;161-1999;242-2000; 323;2001

'2001'

In [14]:
dados = [[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [2,
  1.9100808670106195,
  184.0,
  96331.0,
  96239.0,
  6772088.110875723,
  70.30019527333593,
  '1998'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998']]

In [15]:
dados

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [2,
  1.9100808670106195,
  184.0,
  96331.0,
  96239.0,
  6772088.110875723,
  70.30019527333593,
  '1998'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998']]

In [22]:
dados[0:3]

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [2,
  1.9100808670106195,
  184.0,
  96331.0,
  96239.0,
  6772088.110875723,
  70.30019527333593,
  '1998']]

In [3]:
valor = [80, 57.102130486807575, 2806.325798363425, 49145.72844198477, 47742.56554280306, 461002.36896947364, 9.380314090036832, '2015']

In [4]:
valor

[80,
 57.102130486807575,
 2806.325798363425,
 49145.72844198477,
 47742.56554280306,
 461002.36896947364,
 9.380314090036832,
 '2015']

In [5]:
dados[2] = valor

In [6]:
dados

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [80,
  57.102130486807575,
  2806.325798363425,
  49145.72844198477,
  47742.56554280306,
  461002.36896947364,
  9.380314090036832,
  '2015'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998']]

In [8]:
dados.extend(valor)

In [9]:
dados

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [80,
  57.102130486807575,
  2806.325798363425,
  49145.72844198477,
  47742.56554280306,
  461002.36896947364,
  9.380314090036832,
  '2015'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998'],
 80,
 57.102130486807575,
 2806.325798363425,
 49145.72844198477,
 

In [10]:
dados.append(valor)

In [11]:
dados

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [80,
  57.102130486807575,
  2806.325798363425,
  49145.72844198477,
  47742.56554280306,
  461002.36896947364,
  9.380314090036832,
  '2015'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998'],
 80,
 57.102130486807575,
 2806.325798363425,
 49145.72844198477,
 

In [13]:
dados

[[0,
  33.24,
  3324.0,
  100000.0,
  97154.0,
  6965745.610875723,
  69.65745610875723,
  '1998'],
 [1,
  3.5686209607348256,
  345.0,
  96676.0,
  96503.5,
  6868591.610875723,
  71.04753621245938,
  '1998'],
 [80,
  57.102130486807575,
  2806.325798363425,
  49145.72844198477,
  47742.56554280306,
  461002.36896947364,
  9.380314090036832,
  '2015'],
 [3,
  1.3936992313852747,
  134.0,
  96147.0,
  96080.0,
  6675849.110875723,
  69.43377443784749,
  '1998'],
 [4,
  1.0311103704706654,
  99.0,
  96013.0,
  95958.0,
  6579769.110875723,
  68.52998146996472,
  '1998'],
 [5,
  0.6727213955767808,
  64.516,
  95903.0,
  95870.742,
  6483811.110875723,
  67.60801133307324,
  '1998'],
 [6,
  0.5156592418552863,
  49.42,
  95838.484,
  95813.774,
  6387940.368875723,
  66.65318671855998,
  '1998'],
 [7,
  0.3965379596986144,
  37.983999999999995,
  95789.064,
  95770.072,
  6292126.594875723,
  65.6873168201719,
  '1998'],
 80,
 57.102130486807575,
 2806.325798363425,
 49145.72844198477,
 

In [ ]:

# Manter o vetor até 79 anos. 
# a linha terá a idade x = 80, as outras colunas dentro do for seguirão as fórmulas nos passos
# Obervar os passos 3 e 6, onde tem a variável FA=Fator de ajuste para idade acima de 80 anos
# e o fx = fator de separação de mortes, respectivamente.
# imprimir o fx para a idade 0 (as mortes não se distribuem uniformemente) tem o cálculo (L0 - l1)/d0
# Para as outras idades fx=0.5




In [ ]:
# código para implementar o prolongamento da curva pela técnica da Nota do IBGE
# Extrapolada para as idades acima de 80 anos
# que poderão ser utilizadas nas avaliações atuariais, desde que na projeção dos aspectos
# biométricos dos segurados e de seus dependentes a utilização destas tábuas não indiquem 
# obrigações inferiores às alcançadas com o uso da tábua
# Critérios:
# FA = Fator de Ajuste. começar com 100
# fx = Fator de separação de mortes = 0.5
# Fazer um atingir meta, na expectativa de vida, de modo que fique igual ao divulgado
#  pelo IBGE, alternando a célula do FA - Fator de Ajuste
# Em 2015 a expectativa de vida aos 80 foi de:9,3803140900368 
# Diferença min = E[80] - Ecalc[80], onde E[80] é a experança de vida aos 80, da tabela não extrapolada
# e Ecalc[80] é a nova experança de vida (calculo abaixo, no fim)
# Colunas anteriores + coluna fx, onde fx=0.5
# qx' = prob morte/1000
# prob morte = é o qx da tabela. Na idade 80, onde, na 
# tabua do IBGE é a última idade, embora se viva bem mais do que 80 hoje em dia
# Fazer a interpolação, conforme a nota. = dx/lx*1000 
# dx = lx - lx+1, ou no caso, l80 - l81.
# lx = l[x-1] - d[x-1]
# Lx = fx * lx+(1+fx)*l[x+1]
# Tx = somatorio Lx até l[inf]
# Ex = Tx / lx


In [152]:
# Não aplicar
# Problema de leitura: Tratamento da idade 80+ para os anos. Aqui esta ok
# df_ambos[df_ambos['x'].astype(str).str.contains("80")]

In [153]:
# Não aplicar
# Problema de leitura: Tratamento da idade 80+ para os anos
# df_mulher[df_mulher['x'].astype(str).str.contains("80")].sort_values(by='Ano')

In [154]:
# Problema de leitura: Tratamento da idade 80+ para os anos
# df_homem[df_homem['x'].astype(str).str.contains("80")].sort_values(by='Ano')

In [157]:
# df_mulher[df_mulher['x'].astype(str).str.contains("80")].sort_values(by='Ano')

In [160]:
# Problema de leitura: Tratamento da idade 80+ para os anos
# df_homem[df_homem['x'].astype(str).str.contains("80")].sort_values(by='Ano')

In [145]:
# verificação parcial da base
# df_mulher[df_mulher['Ano']=='2003'][30:81]

In [ ]:
# Código para prolongar a curva com uso da regressão da regressão linear.


In [ ]:
# Graficos com projecoes


In [ ]:
# projeções com Redes Neurais e o uso da regressão linear como baseline

In [ ]:
# Código projeção CNN + LSTM das curva futuras até 2060 anos 

In [ ]:
# Limitação do Estudo: efeitos da pandemia na mortalidade geral, etc

In [ ]:
# Conclusão